# 1. Przygotowanie Darknetu
Darknet jest to opensourceowy framework do realizacji sieci CNN. Pozwala ona na trenowanie sieci na GPU, co diametralnie zmniejsza czas trenowania w porównaniu do CPU.

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14639 (delta 4), reused 8 (delta 3), pack-reused 14621
Receiving objects: 100% (14639/14639), 13.25 MiB | 21.72 MiB/s, done.
Resolving deltas: 100% (9959/9959), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
# make darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

# 2. Google Drive
Łączenie dysku google do środowiska Google Colab i tworzenie symbolicznego linku aby w prosty sposób odwoływać się do położenia dysku.

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive
%cd /content/darknet

/content
Mounted at /content/gdrive


# 3. Dane do trenowania

In [ ]:
!cp /mydrive/data_for_train/coins/obj.zip ../

In [ ]:
!unzip ../obj.zip -d data/

Archive:  ../obj.zip
   creating: data/obj/
  inflating: data/obj/20201211_200236.txt  
  inflating: data/obj/classes.txt    
  inflating: data/obj/IMG_20201211_190633.jpg  
  inflating: data/obj/IMG_20201211_190633.txt  
  inflating: data/obj/IMG_20201211_190708.jpg  
  inflating: data/obj/IMG_20201211_190708.txt  
  inflating: data/obj/IMG_20201211_190736.jpg  
  inflating: data/obj/IMG_20201211_190736.txt  
  inflating: data/obj/IMG_20201211_190810.jpg  
  inflating: data/obj/IMG_20201211_190810.txt  
  inflating: data/obj/IMG_20201211_191547.jpg  
  inflating: data/obj/IMG_20201211_191547.txt  
  inflating: data/obj/IMG_20201211_191710.jpg  
  inflating: data/obj/IMG_20201211_191710.txt  
  inflating: data/obj/IMG_20201211_191729.jpg  
  inflating: data/obj/IMG_20201211_191729.txt  
  inflating: data/obj/IMG_20201211_191816.jpg  
  inflating: data/obj/IMG_20201211_191816.txt  
  inflating: data/obj/IMG_20201211_191835.jpg  
  inflating: data/obj/IMG_20201211_191835.txt  
  inflatin

# 4. Pliki konfiguracyjne
W plikach tych zawarte są parametry modelu oraz spis danych wraz z lokalizacjami danych do trenowania.

In [ ]:
!cp /mydrive/data_for_train/yolov3_custom.cfg ./cfg
!cp /mydrive/data_for_train/obj.names ./data
!cp /mydrive/data_for_train/obj.data  ./data\

YOLOv3 wymaga pliku .txt w którym będą zapisane lokalizacje poszczególnych zdjęć do trenowania sieci. Aby ułatwić sobie zadanie, ten plik jest generowany za pomocą skryptu napisanego w Pythonie.

In [ ]:
!cp /mydrive/data_for_train/generate_train.py ./
!python generate_train.py

# 5. Trenowanie
Aby skrócić czas trenowania można pobrać wstępnie przetrenowane wagi, które będą następnie "dotrenowanie" na własnym datasecie.

In [ ]:
!wget http://pjreddie.com/media/files/darknet53.conv.74

--2020-12-17 01:08:21--  http://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/darknet53.conv.74 [following]
--2020-12-17 01:08:22--  https://pjreddie.com/media/files/darknet53.conv.74
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  9.41MB/s    in 17s     

2020-12-17 01:08:39 (9.23 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



W plikach konfiguracyjnych zapisana jest ścieżka do lokalizacji z backupem sieci. To znaczy, że co 100 epok, wagi sieci są zapisywane na dysku Google, dzięki czemu w razie przerwania trenowania, można je wznowić od ostatniego zapisanego backupu. Finala lokalizacja zapisu wag, również jest ustawiona w plikach konfiguracyjnych.

In [ ]:
# train your custom detector
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 1846817, rewritten_bbox = 0.002491 % 

 1926: 0.199861, 0.229428 avg loss, 0.000010 rate, 11.958452 seconds, 123264 images, 0.549607 hours left
Loaded: 0.000037 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.800618), count: 17, class_loss = 0.453979, iou_loss = 0.782048, total_loss = 1.236028 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.932092), count: 30, class_loss = 0.000243, iou_loss = 0.092056, total_loss = 0.092299 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.931573), count: 16, class_loss = 0.091391, iou_loss = 0.055132, total_loss = 0.146523 
 total_bbox = 1846880, rewritten_bbox = 0.002491 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, 